In [1]:
import os
import pickle
from contextlib import nullcontext
import torch
import tiktoken
from model import GPTConfig, GPT, MemoryGPT

from my_configuration_roberta import MemoryRobertaConfig
from my_modeling_roberta import MemoryRobertaModel

os.environ['TIKTOKEN_CACHE_DIR']="/data/yuanhang/tiktoken_cache_dir"

In [2]:
init_from = 'gpt2' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = 'out' # ignored if init_from is not 'resume'
start = "" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples = 2 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cuda:3' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster

# exec(open('configurator.py').read()) # overrides from command line or config file

In [3]:
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

In [4]:
# model
if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    gptconf = GPTConfig(**checkpoint['model_args'])
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
elif init_from.startswith('gpt2'):
    # init from a given GPT-2 model
    model = MemoryGPT.from_pretrained(init_from, dict(dropout=0.0))

loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rate to 0.0
number of parameters: 123.65M


In [5]:
model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

# look for the meta pickle in case it is available in the dataset folder
load_meta = False
if init_from == 'resume' and 'config' in checkpoint and 'dataset' in checkpoint['config']: # older checkpoints might not have these...
    meta_path = os.path.join('data', checkpoint['config']['dataset'], 'meta.pkl')
    load_meta = os.path.exists(meta_path)
if load_meta:
    print(f"Loading meta from {meta_path}...")
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    # TODO want to make this more general to arbitrary encoder/decoder schemes
    stoi, itos = meta['stoi'], meta['itos']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])
else:
    # ok let's assume gpt-2 encodings by default
    print("No meta.pkl found, assuming GPT-2 encodings...")
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)

# encode the beginning of the prompt
if start.startswith('FILE:'):
    with open(start[5:], 'r', encoding='utf-8') as f:
        start = f.read()

No meta.pkl found, assuming GPT-2 encodings...


# Load Memory

In [6]:
ckpt_path = os.path.join(out_dir, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
evolver_config = MemoryRobertaConfig(vocab_size=checkpoint['model_args']['vocab_size'] + 20, num_hidden_layers=6,
                                     num_attention_heads=12, hidden_size=768, max_position_embeddings=512, intermediate_size=3072,
                                     pad_token_id=0, gpt2_token_id_offset=20, num_memory=10,
                                     num_target_model_layer=12)
evolver_model = MemoryRobertaModel(evolver_config)
state_dict = checkpoint['evolver_model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
evolver_model.load_state_dict(state_dict)

number of parameters: 88.63M


<All keys matched successfully>

In [7]:
evolver_model.eval()
evolver_model.to(device)

MemoryRobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50277, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768, padding_idx=0)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-5): 6 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dr

In [46]:
# generate input memory

context = ["i am a handsome chinese boy. I live in Beijing now.", "i am a handsome chinese boy. I live in Beijing now.", "i am a handsome chinese boy. I live in Beijing now.", "i am a handsome chinese boy. I live in Beijing now."]
# context = ["i am a handsome chinese boy. I live in Beijing now."]
# context = ["i am a handsome chinese boy."]

encoded_context = []

for c in context:
    encoded_context.append(torch.tensor(encode(c))[None, ...].to(device) + 20)

input_memory = None
for index, ec in enumerate(encoded_context):
    output = evolver_model(input_ids=ec, input_memory=input_memory)
    input_memory = output["memory_output"]
    
    if index == 0:
        # print(output["last_hidden_state"])
        # print(input_memory)
        pass

print(input_memory)

target_model_memory = evolver_model(input_memory=input_memory, produce_memory_flag=True)
print(target_model_memory.shape)

tensor([[[ 1.1861,  1.1499,  0.1170,  ..., -0.6563, -0.3641, -0.5765],
         [ 1.1784,  1.1462,  0.1218,  ..., -0.6369, -0.3493, -0.5873],
         [ 1.2003,  1.1558,  0.1102,  ..., -0.6811, -0.3793, -0.5673],
         ...,
         [ 1.1719,  1.1428,  0.1269,  ..., -0.6271, -0.3427, -0.5891],
         [ 1.1809,  1.1463,  0.1191,  ..., -0.6475, -0.3576, -0.5813],
         [ 1.1782,  1.1464,  0.1197,  ..., -0.6444, -0.3568, -0.5823]]],
       device='cuda:3', grad_fn=<SliceBackward0>)
torch.Size([12, 1, 10, 768])


# Inference

In [48]:
question = "i am a "

start_ids = encode(start + question)
x = torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...]
print(x)

# run generation
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k, input_memory=target_model_memory)

            result = y[0].tolist()

            eot_index = len(result)
            for ci, c in enumerate(result):
                if c == enc.eot_token:
                    eot_index = ci
                    break
            
            print(decode(result[:eot_index]))
            print('---------------')

tensor([[ 72, 716, 257, 220]], device='cuda:3')
i am a vernacular the world over. . The major media doesn’t do the business of the world. The Internet is a fucking internet. I don’t care how many people I know have commented on this. My information is in the public domain. I don’t care. It’s my life. I don’t care. I just need to be polite and helpful. I do not care about personal info, I’m absolutely fine with that. When I’m just beginning, I want to get a nudge. I want to let people know how they’ll be able to decide what they want and what they want to see of me.
---------------
i am a ursa. As a saur on my own I am not raved about, but I am pleased with how well it is going. I got a 9.5 on a custom motorcycle and the new one is a 9.9. The bike is good so far, and the engine is quiet, but I am not a fan of the turbocharger on the new bike. The throttle is pretty much the same as the old one, with all fours, but they are a bit out of balance. The throttle is good so far, and the fuel 

In [26]:
question = "i am a handsome chinese boy. I live"

start_ids = encode(start + question)

print(start_ids)

my_ids = [72, 716, 257, 22665, 442, 3762, 2933, 13, 314, 2107]

decode(my_ids)

my_ids = [72, 716, 257, enc.eot_token, 442, 3762, 2933, 13, 314, 2107]

decode(my_ids)

[72, 716, 257, 22665, 442, 3762, 2933, 13, 314, 2107]


'i am a<|endoftext|> chinese boy. I live'